In [109]:
"""dataset = https://www.kaggle.com/datasets/mastmustu/weather-analysis"""

'dataset = https://www.kaggle.com/datasets/mastmustu/weather-analysis'

In [110]:
import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf



In [111]:
data = pd.read_csv('climate_data.csv')
data.head()

,Date,Average temperature (°F),Average humidity (%),Average dewpoint (°F),Average barometer (in),Average windspeed (mph),Average gustspeed (mph),Average direction (°deg),Rainfall for month (in),Rainfall for year (in),...,Maximum humidity (%),Minimum humidity (%),Maximum pressure,Minimum pressure,Maximum windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F),Date1,Month,diff_pressure
0,2009-01-01,37.8,35.0,12.7,29.7,26.4,36.8,274.0,0.0,0.0,...,4.0,27.0,29.762,29.596,41.4,59.0,40.0,2009-01-01,1,0.166
1,2009-01-02,43.2,32.0,14.7,29.5,12.8,18.0,240.0,0.0,0.0,...,4.0,16.0,29.669,29.268,35.7,51.0,52.0,2009-01-02,1,0.401
2,2009-01-03,25.7,60.0,12.7,29.7,8.3,12.2,290.0,0.0,0.0,...,8.0,35.0,30.232,29.260,25.3,38.0,41.0,2009-01-03,1,0.972
3,2009-01-04,9.3,67.0,0.1,30.4,2.9,4.5,47.0,0.0,0.0,...,7.0,35.0,30.566,30.227,12.7,20.0,32.0,2009-01-04,1,0.339
4,2009-01-05,23.5,30.0,-5.3,29.9,16.7,23.1,265.0,0.0,0.0,...,5.0,13.0,30.233,29.568,38.0,53.0,32.0,2009-01-05,1,0.665


In [112]:
data.isnull().sum()       # cek adakah nilai yang hilang

Date                        0
Average temperature (°F)    0
Average humidity (%)        0
Average dewpoint (°F)       0
Average barometer (in)      0
Average windspeed (mph)     0
Average gustspeed (mph)     0
Average direction (°deg)    0
Rainfall for month (in)     0
Rainfall for year (in)      0
Maximum rain per minute     0
Maximum temperature (°F)    0
Minimum temperature (°F)    0
Maximum humidity (%)        0
Minimum humidity (%)        0
Maximum pressure            0
Minimum pressure            0
Maximum windspeed (mph)     0
Maximum gust speed (mph)    0
Maximum heat index (°F)     0
Date1                       0
Month                       0
diff_pressure               0
dtype: int64

In [113]:
dates = data['Date'].values
temp  = data['Average temperature (°F)'].values

threshold = (temp.max() - temp.min()) * 10/100      #tresshold
print(threshold)

8.839999999999998


**bagi data test dan train**

In [114]:
# MEMBAGI data test dan train
# https://stats.stackexchange.com/questions/346907/splitting-time-series-data-into-train-test-validation-sets
# apakah data dibagi memakai train test split? bukankah data berhubungan dengan series sehingga tidak bisa diacak?
print(len(temp))
temp_train = temp[0:3220]         # 80% dari 3902 data = 3220 data  
temp_test = temp[3220:len(temp)]  # 20% dari 3902 data = 3902-3220 data 

print(temp_train)
print(temp_test)

3902
[37.8 43.2 25.7 ... 64.  67.9 66.1]
[55.1 59.  61.7 61.  61.2 56.4 56.  54.  53.8 51.8 53.  57.7 62.8 59.5
 73.4 69.1 63.1 64.8 64.8 62.4 61.  60.1 50.2 55.6 57.5 55.5 47.6 48.1
 58.  39.4 57.6 53.7 56.7 60.4 59.8 48.6 50.  38.7 38.6 32.6 31.2 26.5
 33.3 44.1 46.3 17.9 19.9 31.1 34.7 40.3 44.9 40.7 48.9 45.3 43.8 45.9
 46.7 48.4 53.3 49.4 56.1 31.5 30.8 36.5 43.6 35.8 36.1 34.5 30.1 26.1
 34.6 17.3 14.2 41.3 41.3 20.6 22.  31.2 33.2 34.8 40.9 34.8 30.8 24.8
 32.6 39.1 39.5 28.4 20.4 12.7 18.8 40.  39.5 36.1 33.8 38.2 42.6 39.7
 37.3 38.7 30.7 30.7 38.5 28.  31.7 32.1 24.  23.4 15.4 10.2 18.9 30.8
  9.5  2.4 13.7 31.9 44.  43.7 36.2 30.1 33.1 26.7 16.9 18.2 33.1 32.9
 38.4 30.2 31.6 40.8 36.2 20.4 23.5 21.1 24.7 29.2 35.5 21.3 25.8 31.9
 36.6 40.  41.5 42.3 38.5 36.5 13.9  6.9 24.3 25.9 27.4 23.8 28.7 36.8
 33.3 35.3 25.2 16.7  7.9  7.4 13.9 13.5 18.5 20.9 26.8 35.8 42.7 41.4
 38.3 11.6  1.   5.9 27.  36.6 36.7 33.6 28.7 28.3 29.8 38.5 24.1 22.5
 24.1 26.8 30.8 30.6 30.3 31.4 33.9 

In [116]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):#mengubah data jadi format yg bisa diterima model
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

In [117]:
train_temp = windowed_dataset(temp_train, window_size=16, batch_size=100, shuffle_buffer=1000)
test_temp = windowed_dataset(temp_test, window_size=16, batch_size=100, shuffle_buffer=1000)

print(train_temp)
print(test_temp)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>


In [118]:
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(32),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(32, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
])

**Tresshold**

In [115]:
print(threshold)

8.839999999999998


In [119]:
optimizer = tf.keras.optimizers.SGD(lr=1.0000e-04, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_temp,epochs=100)

Epoch 1/100


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


33/33 [==============================] - 6s 41ms/step - loss: 44.7098 - mae: 45.2096
Epoch 2/100
33/33 [==============================] - 1s 40ms/step - loss: 44.5702 - mae: 45.0700
Epoch 3/100
33/33 [==============================] - 1s 40ms/step - loss: 44.5021 - mae: 45.0019
Epoch 4/100
33/33 [==============================] - 1s 40ms/step - loss: 44.4324 - mae: 44.9322
Epoch 5/100
33/33 [==============================] - 1s 40ms/step - loss: 44.3581 - mae: 44.8578
Epoch 6/100
33/33 [==============================] - 1s 40ms/step - loss: 44.2749 - mae: 44.7747
Epoch 7/100
33/33 [==============================] - 1s 40ms/step - loss: 44.1843 - mae: 44.6840
Epoch 8/100
33/33 [==============================] - 1s 39ms/step - loss: 44.0816 - mae: 44.5814
Epoch 9/100
33/33 [==============================] - 1s 40ms/step - loss: 43.9656 - mae: 44.4653
Epoch 10/100
33/33 [==============================] - 1s 40ms/step - loss: 43.8232 - mae: 44.3230
Epoch 11/100
33/33 [=====================

In [120]:
model.evaluate(test_temp)

7/7 [==============================] - 1s 21ms/step - loss: 5.2577 - mae: 5.7417


[5.2577033042907715, 5.741715431213379]